In [116]:
import utils
import lizard
from diff_match_patch import diff_match_patch

In [117]:
# in our example out of repo code corresponds to before code
# and the target repo code corresponds to the after code

In [118]:
before_src = None
after_src = None

with open('before_functions.py', 'r') as file:
    before_src = file.read()

with open('after_functions.py', 'r') as file:
    after_src = file.read()

In [119]:
# get the lizard func objects
before_funcs = lizard.analyze_file.analyze_source_code('before_functions.py', before_src).function_list
after_funcs = lizard.analyze_file.analyze_source_code('after_functions.py', after_src).function_list

func_pairs = list(zip(before_funcs, after_funcs))

func_pairs

[(<lizard.FunctionInfo at 0x7f3e10371610>,
  <lizard.FunctionInfo at 0x7f3e103c30a0>),
 (<lizard.FunctionInfo at 0x7f3e10371a60>,
  <lizard.FunctionInfo at 0x7f3e103c3bb0>),
 (<lizard.FunctionInfo at 0x7f3e10371b80>,
  <lizard.FunctionInfo at 0x7f3e103c3490>)]

In [120]:
modified_funcs_list = []

for pair in func_pairs:
    modified_func = utils.ModifiedFunc(pair[0], pair[1], before_src, after_src)
    modified_funcs_list.append(modified_func)
    print(modified_func.to_json())

{'func_name': 'func1', 'func_src_before': 'def func1():\n    #line 1\n    #line 2\n    #line 3\n    #line 4\n    #line 5\n    pass', 'func_src_after': "def func1():\n    #line 1\n    #line and then comething else\n    print('nothing')\n    pass", 'line_changes': {'deleted': [], 'added': []}, 'char_changes': {'deleted': [{'char_start': 35, 'char_end': 72, 'chars': '2\n    #line 3\n    #line 4\n    #line 5'}], 'added': [{'char_start': 35, 'char_end': 79, 'chars': "and then comething else\n    print('nothing')"}]}}
{'func_name': 'func2', 'func_src_before': 'def func2():\n    #line 1\n    #line 2\n    #line 3\n    #line 4\n    #line 5\n    pass', 'func_src_after': 'def func2():\n    #line 0\n    #line 1\n    #line 2\n    #line 3\n    #line 5\n    #line 6\n    #line 7\n    pass', 'line_changes': {'deleted': [], 'added': []}, 'char_changes': {'deleted': [{'char_start': 59, 'char_end': 60, 'chars': '4'}, {'char_start': 71, 'char_end': 72, 'chars': '5'}], 'added': [{'char_start': 13, 'char_end

In [121]:
for f in modified_funcs_list:
    print(f.char_changes)
    print(f.line_changes)
    print()

{'deleted': [{'char_start': 35, 'char_end': 72, 'chars': '2\n    #line 3\n    #line 4\n    #line 5'}], 'added': [{'char_start': 35, 'char_end': 79, 'chars': "and then comething else\n    print('nothing')"}]}
{'deleted': [], 'added': []}

{'deleted': [{'char_start': 59, 'char_end': 60, 'chars': '4'}, {'char_start': 71, 'char_end': 72, 'chars': '5'}], 'added': [{'char_start': 13, 'char_end': 25, 'chars': '    #line 0\n'}, {'char_start': 71, 'char_end': 84, 'chars': '5\n    #line 6'}, {'char_start': 95, 'char_end': 96, 'chars': '7'}]}
{'deleted': [], 'added': []}

{'deleted': [], 'added': []}
{'deleted': [], 'added': []}



In [122]:
def create_deletions(modified_func):

    before_code = modified_func.func_src_before
    before_code_lines = before_code.split('\n')

    # for every entry in deleted and added, extract the code and indeces
    # then count how many new lines come before
    # then check to see if every line in the changed code portion matches completely to an entry in
    # the corresponding source code, and add to either an addition or deletion

    # deletetions will compare to the before code
    # additions will compare to the after code

    char_changes_delete = modified_func.char_changes['deleted']
    for change in char_changes_delete:

        # count how many lines happened before we started changing code
        # then add 1 to describe the current line where we changed code
        start_line_num = before_code[:change['char_start']].count('\n') + 1

        # retrieve the changed code, and split each into their own line
        # fileter out the empty line
        changed_code = before_code[change['char_start']:change['char_end']+1].split('\n')
        changed_code = list(filter(None, changed_code))

        for l in range(len(changed_code)):
            # line num is 1 indexed
            curr_line_num = start_line_num + l

            # access the lines to analyze form the original and 
            # characters which have been deleted from
            changed_line = changed_code[l]
            src_code_line = before_code_lines[curr_line_num - 1]

            # if the characters changed on the current line match extactly up with 
            # the entirety of the line in the src code then we can count this as 
            # a complete line deletion
            if changed_line.strip() == src_code_line.strip():

                # get the indeces of the current line that has been deleted
                start_index = len('\n'.join(before_code_lines[:curr_line_num - 1])) + 1
                end_index = start_index + len(src_code_line)

                deletion = {"line_no": curr_line_num, "char_start": start_index, "char_end": end_index, "line": src_code_line}
                modified_func.line_changes['deleted'].append(deletion)

                

In [123]:
def create_additions(modified_func):
    pass

In [129]:

# using the character changes and the function source code
# determine which of the character changes constitute a complete line addition or deletion
def get_line_changes(modified_func):

    # create_deletions(modified_func)
    create_additions(modified_func)



In [130]:
get_line_changes(modified_funcs_list[0])

In [131]:
modified_funcs_list[0].to_json()

{'func_name': 'func1',
 'func_src_before': 'def func1():\n    #line 1\n    #line 2\n    #line 3\n    #line 4\n    #line 5\n    pass',
 'func_src_after': "def func1():\n    #line 1\n    #line and then comething else\n    print('nothing')\n    pass",
 'line_changes': {'deleted': [{'line_no': 4,
    'char_start': 37,
    'char_end': 48,
    'line': '    #line 3'},
   {'line_no': 5, 'char_start': 49, 'char_end': 60, 'line': '    #line 4'},
   {'line_no': 6, 'char_start': 61, 'char_end': 72, 'line': '    #line 5'}],
  'added': []},
 'char_changes': {'deleted': [{'char_start': 35,
    'char_end': 72,
    'chars': '2\n    #line 3\n    #line 4\n    #line 5'}],
  'added': [{'char_start': 35,
    'char_end': 79,
    'chars': "and then comething else\n    print('nothing')"}]}}